# Illustration of Method of Images

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from brainsignals.plotting_convention import mark_subplots, cmap_v_e

In [ ]:
def point_source_potential(elec_locs, sources_pos, imem):

    num_elecs = elec_locs.shape[1]
    num_sources = sources_pos.shape[1]
    r2 = np.zeros((num_elecs, num_sources))
    for elec in range(num_elecs):
        for s in range(num_sources):
            r2[elec, s] = np.sum([(sources_pos[n, s] - elec_locs[n, elec])**2
                                 for n in range(3)])

    mapping = 1 / (4 * np.pi * sigma * np.sqrt(r2))
    v_e = mapping @ imem * 1000
    return v_e

In [ ]:

h = 100
sigma = 0.3 # s/m
source_pos = np.array([0, 0, -h])
mirror_pos = np.array([0, 0, h])

imem_1 = np.array([1., 1])
imem_2 = np.array([1., -1])

sources_pos = np.array([source_pos, mirror_pos]).T

fig = plt.figure(figsize=[6, 2.2])

plt.subplots_adjust(hspace=0.3, wspace=0.1, right=0.98, left=0.05, top=0.98, bottom=0.02)

ax1 = fig.add_subplot(121, aspect=1, frameon=False,
                   xticks=[], yticks=[], title=r"$\sigma_1 = 0.3$ S/m; $\sigma_2 = 0$")
ax2 = fig.add_subplot(122, aspect=1, frameon=False,
                   xticks=[], yticks=[], title=r"$\sigma_1 = 0.3$ S/m; $\sigma_2 = \infty$")


ax1.plot(source_pos[0], source_pos[2], 'o', ms=5, color='k', zorder=1)
ax1.plot(mirror_pos[0], mirror_pos[2], 'o', ms=5, color='k', zorder=1)

ax2.plot(source_pos[0], source_pos[2], 'o', ms=5, color='k', zorder=1)
ax2.plot(mirror_pos[0], mirror_pos[2], 'o', ms=5, color='k', zorder=1)

grid_x, grid_z = np.mgrid[-501:501:5, -350:351:5]
grid_y = np.zeros(grid_x.shape)

electrode_grid = np.array([grid_x.flatten(),
                           grid_y.flatten(),
                           grid_z.flatten()])

V_e_1 = point_source_potential(electrode_grid, sources_pos, imem_1)
V_e_2 = point_source_potential(electrode_grid, sources_pos, imem_2)

V_e_1 = V_e_1.reshape(grid_x.shape)
V_e_2 = V_e_2.reshape(grid_x.shape)

num = 15
levels = np.logspace(-3, 0, num=num)

scale_max = np.max(np.abs(V_e_1))
print(scale_max)

levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2))
                   for i in range(len(levels_norm) -1)]
colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)


ep_2m = ax1.contourf(grid_x, grid_z, V_e_1,
                           zorder=-2, colors=colors_from_map,
                           levels=levels_norm, extend='both')
ep_2m = ax1.contour(grid_x, grid_z, V_e_1,
                           zorder=-2, colors='k', linewidths=1,
                           levels=levels_norm, extend='both')

ep_2m = ax2.contourf(grid_x, grid_z, V_e_2,
                           zorder=-2, colors=colors_from_map,
                           levels=levels_norm, extend='both')
ep_2m = ax2.contour(grid_x, grid_z, V_e_2,
                           zorder=-2, colors='k', linewidths=1,
                           levels=levels_norm, extend='both')
for ax in [ax1, ax2]:
    ax.plot([-500, 500], [0, 0], ls='--', c='gray')
    ax.plot([530, 530], [0, h], '-', marker='_', lw=2, c='k',solid_capstyle='butt')
    ax.plot([530, 530], [0, -h], '-', marker='_', lw=2, c='k',solid_capstyle='butt')
    ax.text(540, h/2, "$h$", va='center')
    ax.text(540, -h/2, "$h$", va='center')
    ax.text(-550, -h/2, "$\sigma_1$", va='center')
    ax.text(-550, h/2, "$\sigma_2$", va='center')
mark_subplots([ax1, ax2], xpos=-0.05, ypos=1.05)
fig.savefig("mirror_point_source_illustration.pdf")
